# Demo

let's analyze few homes



### step 1: who might be eligible 

Santa Clara County 车库改造 + detached adu

Alameda County - 车库改造 + detached adu

la county - 车库改造最多

San Francisco County - 车库改造居多

### find price per sqft high and total price not high
# lot size / home size; 
total price / price per sqft





In [ ]:
# Read JSON data from county files
import json
import pandas as pd
import os

#display width change to 1000
pd.set_option('display.max_colwidth', 200)



In [ ]:
# Define file paths
data_dir = "../data/raw/"
county_files = {
    "Santa Clara": "20250822_114114_santa_clara_county_ca_santa_clara_county.json",
    "Alameda": "20250822_114052_alameda_county_ca_alameda_county.json", 
    "Los Angeles": "20250822_113940_los_angeles_county_ca_los_angeles_county.json"
}

# Load data from each county
county_data = {}
for county_name, filename in county_files.items():
    filepath = os.path.join(data_dir, filename)
    with open(filepath, 'r') as f:
        data = json.load(f)
        county_data[county_name] = data
        print(f"Loaded {county_name} County: {len(data['listings'])} listings")

# Convert listings to DataFrames for easier analysis
county_dfs = {}
for county_name, data in county_data.items():
    # Extract relevant fields from each listing
    listings = []
    for listing in data['listings']:
        listing_data = {
            'address': listing.get('location', {}).get('address', {}).get('line', ''),
            'city': listing.get('location', {}).get('address', {}).get('city', ''),
            'zip': listing.get('location', {}).get('address', {}).get('postal_code', ''),
            'price': listing.get('list_price', 0),
            'price_per_sqft': listing.get('price_per_sqft', 0),
            'sqft': listing.get('description', {}).get('sqft', 0),
            'lot_sqft': listing.get('description', {}).get('lot_sqft', 0),
            'beds': listing.get('description', {}).get('beds', 0),
            'baths': listing.get('description', {}).get('baths', 0),
            'description': listing.get('description', {}).get('text', ''),
            'year_built': listing.get('description', {}).get('year_built', 0),
            'garage': listing.get('description', {}).get('garage', 0),
            'property_type': listing.get('description', {}).get('type', ''),
            'listing_id': listing.get('listing_id', ''),
            'url': listing.get('href', '')
        }
        listings.append(listing_data)
    
    county_dfs[county_name] = pd.DataFrame(listings)
    print(f"{county_name} DataFrame created with shape: {county_dfs[county_name].shape}")

# Display basic stats for each county
for county_name, df in county_dfs.items():
    print(f"\n{county_name} County Statistics:")
    print(f"  Average price: ${df['price'].mean():,.0f}")
    print(f"  Average price/sqft: ${df['price_per_sqft'].mean():,.0f}")
    print(f"  Average sqft: {df['sqft'].mean():,.0f}")
    print(f"  Average lot sqft: {df['lot_sqft'].mean():,.0f}")

#conver to pandas dataframe
county_dfs = pd.concat(county_dfs.values())

#display
county_dfs.head(5)


Loaded Santa Clara County: 1366 listings
Loaded Alameda County: 1841 listings
Loaded Los Angeles County: 4513 listings
Santa Clara DataFrame created with shape: (1366, 15)
Alameda DataFrame created with shape: (1841, 15)
Los Angeles DataFrame created with shape: (4513, 15)

Santa Clara County Statistics:
  Average price: $2,192,131
  Average price/sqft: $0
  Average sqft: 2,104
  Average lot sqft: 28,001

Alameda County Statistics:
  Average price: $1,557,264
  Average price/sqft: $0
  Average sqft: 2,202
  Average lot sqft: 229,482

Los Angeles County Statistics:
  Average price: $1,346,727
  Average price/sqft: $0
  Average sqft: 2,073
  Average lot sqft: 465,014


,address,city,zip,price,price_per_sqft,sqft,lot_sqft,beds,baths,description,year_built,garage,property_type,listing_id,url
0,5939 Pala Mesa Dr,San Jose,95123,1899888,0,2451.0,4715.0,4,3.0,Welcome to 5939 Pala Mesa Drive in the desirable Palmia community. This 4-be...,2000.0,0,single_family,2985606552,https://www.realtor.com/realestateandhomes-detail/5939-Pala-Mesa-Dr_San-Jose...
1,119 Belcrest Dr,Los Gatos,95032,2699000,0,1748.0,10376.0,4,2.0,Beautifully renovated single-story gem located in the sought-after Belwood o...,1965.0,0,single_family,2985604175,https://www.realtor.com/realestateandhomes-detail/119-Belcrest-Dr_Los-Gatos_...
2,2055 Rucker Ave,Gilroy,95020,1649777,0,1172.0,306662.0,3,2.0,"Brand-new home on 7 flat acres + permits for a 4, 500 sq ft 2nd home! 2 addr...",2025.0,0,single_family,2985604058,https://www.realtor.com/realestateandhomes-detail/2055-Rucker-Ave_Gilroy_CA_...
3,5850 Chesbro Ave,San Jose,95123,1499900,0,1777.0,6000.0,3,3.0,"Welcome to 5850 Chesbro Ave, a beautifully remodeled home in the heart of Bl...",1964.0,0,single_family,2985597086,https://www.realtor.com/realestateandhomes-detail/5850-Chesbro-Ave_San-Jose_...
4,1641 Rue Avati,San Jose,95131,1349888,0,1205.0,6247.0,3,2.0,"Welcome to 1641 Rue Avati, San Jose, CA 95131. This beautifully upgraded Nor...",1973.0,0,single_family,2985596768,https://www.realtor.com/realestateandhomes-detail/1641-Rue-Avati_San-Jose_CA...


In [ ]:
#calculate price per sqft / total price
county_dfs['price_per_sqft'] = county_dfs['price'] / county_dfs['sqft']
print(county_dfs.shape)

#analyze county_dfs['price_per_sqft'] 
#plot price per sqft
# import matplotlib.pyplot as plt
# plt.figure(figsize=(10, 6))
# plt.hist(county_dfs['price_per_sqft'], bins=50, color='skyblue', edgecolor='black')
# plt.title('Distribution of Price per Square Foot')
# plt.xlabel('Price per Square Foot')
# plt.ylabel('Frequency')
# plt.show()

#filter price and unit price
county_dfs = county_dfs[(county_dfs['price']<3000000)&(county_dfs['price']>700000)]
print(county_dfs.shape)

county_dfs = county_dfs[(county_dfs['price_per_sqft']>600)&(county_dfs['price_per_sqft']<1200)]
print(county_dfs.shape)
#calculate  lot sqft/home sqft ratio
county_dfs['lot_sqft_ratio'] = county_dfs['lot_sqft'] / county_dfs['sqft']
print(county_dfs.shape)

#remove those who have lot_sqft_ratio > 100
county_dfs = county_dfs[(county_dfs['lot_sqft_ratio']<20)&(county_dfs['lot_sqft_ratio']>3)]
print(county_dfs.shape)
#select lot sqft > 5000 and home sqft < 2500
county_dfs = county_dfs[(county_dfs['lot_sqft']>5000)&(county_dfs['sqft']<2500)]
print(county_dfs.shape)
#select home year built < 2006
county_dfs = county_dfs[county_dfs['year_built']<2010]
print(county_dfs.shape)


#analyze lot_sqft_ratio
#plot lot_sqft_ratio
# import matplotlib.pyplot as plt
# plt.figure(figsize=(10, 6))
# plt.hist(county_dfs['lot_sqft_ratio'], bins=50, color='skyblue', edgecolor='black')
# plt.title('Distribution of Lot Square Footage Ratio')
# plt.xlabel('Lot Square Footage Ratio')
# plt.ylabel('Frequency')
# plt.show()

county_dfs = county_dfs.reset_index(drop=True)
county_dfs


#remove those who have adu already

(7720, 15)
(7720, 15)
(7362, 15)
(4410, 15)
(4410, 16)
(3205, 16)
(2589, 16)
(2563, 16)


,address,city,zip,price,price_per_sqft,sqft,lot_sqft,beds,baths,description,year_built,garage,property_type,listing_id,url,lot_sqft_ratio
0,5850 Chesbro Ave,San Jose,95123,1499900,844.063028,1777.0,6000.0,3,3.0,"Welcome to 5850 Chesbro Ave, a beautifully remodeled home in the heart of Bl...",1964.0,0,single_family,2985597086,https://www.realtor.com/realestateandhomes-detail/5850-Chesbro-Ave_San-Jose_...,3.376477
1,1641 Rue Avati,San Jose,95131,1349888,1120.239004,1205.0,6247.0,3,2.0,"Welcome to 1641 Rue Avati, San Jose, CA 95131. This beautifully upgraded Nor...",1973.0,0,single_family,2985596768,https://www.realtor.com/realestateandhomes-detail/1641-Rue-Avati_San-Jose_CA...,5.184232
2,5991 Sorrel Ave,San Jose,95123,1600000,819.672131,1952.0,6120.0,5,2.0,"Welcome to 5991 Sorrel Avenue, a rare single-story, five-bedroom residence o...",1965.0,0,single_family,2985592836,https://www.realtor.com/realestateandhomes-detail/5991-Sorrel-Ave_San-Jose_C...,3.135246
3,266 Vista Ave,San Jose,95127,900000,813.008130,1107.0,7000.0,2,1.0,"PLEASE DO NOT DISTURB OCCUPANTS. Charming 2-bedroom home on a spacious 7, 00...",1920.0,0,single_family,2985590778,https://www.realtor.com/realestateandhomes-detail/266-Vista-Ave_San-Jose_CA_...,6.323397
4,2936 Stutz Way,San Jose,95148,1650000,1057.692308,1560.0,6000.0,4,2.0,"Welcome to 2936 Stutz Way, a beautifully updated home in a great Evergreen n...",1976.0,0,single_family,2985588295,https://www.realtor.com/realestateandhomes-detail/2936-Stutz-Way_San-Jose_CA...,3.846154
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2558,525 Hacienda Dr,Monrovia,91016,1125000,1057.330827,1064.0,7410.0,2,1.0,"Welcome to this charming 2 bedroom, 1 bathroom home nestled near the scenic ...",1949.0,0,single_family,2984992140,https://www.realtor.com/realestateandhomes-detail/525-Hacienda-Dr_Monrovia_C...,6.964286
2559,6460 W 84th St,Los Angeles,90045,1595000,1126.412429,1416.0,6105.0,3,2.0,"Do not miss this one, it is fabulous! It really has everything you have been...",1944.0,0,single_family,2984216377,https://www.realtor.com/realestateandhomes-detail/6460-W-84th-St_Los-Angeles...,4.311441
2560,4040 Lehman Rd,La Crescenta,91214,1275000,728.987993,1749.0,8125.0,3,2.0,Great Opportunity in Prime Glendale Location! Located in the highly desirabl...,1956.0,0,single_family,2984221760,https://www.realtor.com/realestateandhomes-detail/4040-Lehman-Rd_La-Crescent...,4.645512
2561,2435 Old Grand St,Santa Ana,92705,950000,611.719253,1553.0,8600.0,3,2.0,"Located in the highly desirable 92705 area of Santa Ana, 2435 Old Grand Stre...",1955.0,0,single_family,2984997579,https://www.realtor.com/realestateandhomes-detail/2435-Old-Grand-St_Santa-An...,5.537669


In [36]:
import re
# 定义ADU相关的关键词模式（不区分大小写）
adu_pattern = re.compile(r'\b(adu|accessory\s+dwelling|granny\s+unit|in-?law\s+unit|secondary\s+unit)\b', re.IGNORECASE)
#add True or False to county_dfs and new column name is existing_adu (1 if adu_pattern is found in description, 0 otherwise)
#handle null values
county_dfs['description'] = county_dfs['description'].fillna(' ')
county_dfs['existing_adu'] = county_dfs['description'].apply(lambda x: 1 if adu_pattern.search(x) is not None else 0)

#print count of existing_adu 1 and 0
print(county_dfs[county_dfs['existing_adu']==0].shape)
print(county_dfs[county_dfs['existing_adu']==1].shape)

#display county_dfs
county_dfs[county_dfs['existing_adu']==1]


(2002, 17)
(561, 17)


,address,city,zip,price,price_per_sqft,sqft,lot_sqft,beds,baths,description,year_built,garage,property_type,listing_id,url,lot_sqft_ratio,existing_adu
7,818 N 13th St,San Jose,95112,1270000,965.045593,1316.0,5375.0,3,1.0,"Charming 3-Bedroom Craftsman in Prime Northside San Jose Location Welcome to this inviting 3-bedroom, 1-bath Craftsman-style home, complete with a dedicated laundry room for added convenience. Nes...",1910.0,0,single_family,2985583896,https://www.realtor.com/realestateandhomes-detail/818-N-13th-St_San-Jose_CA_95112_M27865-91257,4.084347,1
16,771 N 16th St,San Jose,95112,999900,824.319868,1213.0,6625.0,3,2.0,"Exquisitely remodeled Home featuring over $200k of newer updates and renovations. Functional and spacious floor plan offers 3 bedrooms, 2 bathrooms, partial basement is finished and can be used as...",1947.0,0,single_family,2985568233,https://www.realtor.com/realestateandhomes-detail/771-N-16th-St_San-Jose_CA_95112_M13815-04789,5.461665,1
22,3305 Hickerson Dr,San Jose,95127,999900,691.015895,1447.0,5225.0,3,3.0,"Charming Home with ADU & Modern Upgrades! 3 bedrooms, 2.5 bathrooms plus ADU! This 1, 447 sq. ft. home on a 5, 225 sq. ft. lot includes an ADU with bedroom, living area, kitchenette, and full bath...",1962.0,0,single_family,2985545067,https://www.realtor.com/realestateandhomes-detail/3305-Hickerson-Dr_San-Jose_CA_95127_M19058-66865,3.610919,1
27,82 Hollywood Ave,San Jose,95112,1288000,658.486708,1956.0,5922.0,4,2.0,"Offering $10, 000 rate buy-down in one of Downtown San Jose's most sought-after neighborhoods. Over 1900 sq. ft. of interior living space with four bedrooms and two full bathrooms. High-end finish...",1920.0,0,single_family,2985536266,https://www.realtor.com/realestateandhomes-detail/82-Hollywood-Ave_San-Jose_CA_95112_M11013-14762,3.027607,1
39,336 West Ct,San Jose,95116,988000,1020.661157,968.0,5625.0,4,3.0,"Rare, flexible setup with two separate, newly updated residences on a 5625 sf lot. Main House ( 2BD/1BA | 968 sq ft): Classic 1920s charm meets modern updates. Arched openings brighten a smart, si...",1927.0,0,single_family,2985510089,https://www.realtor.com/realestateandhomes-detail/336-West-Ct_San-Jose_CA_95116_M10942-44383,5.810950,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2549,867 New York Dr,Altadena,91001,1400000,664.767331,2106.0,7314.0,4,3.0,"Welcome to 867 New York Drive, a beautifully maintained 1930s Colonial-style home that effortlessly blends timeless elegance with thoughtful modern updates, nestled on one of Altadena's most vibra...",1937.0,0,single_family,2984232934,https://www.realtor.com/realestateandhomes-detail/867-New-York-Dr_Altadena_CA_91001_M21768-77486,3.472934,1
2554,13345 Burbank Blvd,Van Nuys,91401,1100000,947.459087,1161.0,6719.0,3,1.0,"SOLD Before PROCESSING. Decisions, decisions. Sherman Oaks adjacent. There are two homes on this R3 zoned, 6, 716 sqft lot and they have been in one family for 60+ Years. The homes are tired and n...",1948.0,0,single_family,2985005789,https://www.realtor.com/realestateandhomes-detail/13345-Burbank-Blvd_Van-Nuys_CA_91401_M24697-13964,5.787252,1
2555,4803 Gambier St,Los Angeles,90032,729000,767.368421,950.0,7362.0,3,2.0,"Think owning a home in Los Angeles is out of reach? Think again.Welcome to El Sereno, a neighborhood that feels like home the moment you arrive. This sweet bungalow sits on a quiet street, right a...",1923.0,0,single_family,2985005429,https://www.realtor.com/realestateandhomes-detail/4803-Gambier-St_Los-Angeles_CA_90032_M15857-58808,7.749474,1
2557,557 Black Hills Dr,Claremont,91711,900000,604.838710,1488.0,9875.0,4,2.0,"Welcome to a home where ease and enjoyment come together in a beautiful Claremont neighborhood. This thoughtfully updated single-level residence offers comfort, style, and low-maintenance living b...",1962.0,0,single_family,2985004154,https://www.realtor.com/realestateandhomes-detail/557-Black-Hills-Dr_Claremont_CA_91711_M21488-61017,6.636425,1


### Step 2: Check Zone and city code


In [43]:
with open('../data/adu-ca-laws.json', 'r') as f:
    adu_ca_laws = json.load(f)


In [ ]:
# Function to search for ADU codes by city
def get_adu_codes_by_city(city_name, adu_laws):
    """
    Search for ADU codes by city name in the adu_ca_laws data
    Returns a dictionary with ADU information if found
    """
    city_name_lower = city_name.lower().strip()
    # Fix: Convert spaces to underscores to match ADU laws format
    city_key = city_name_lower.replace(' ', '_')
    
    # Search through all counties
    for county_key, county_data in adu_laws['california_adu_regulations'].items():
        if county_key.endswith('_county') and 'cities' in county_data:
            cities = county_data['cities']
            if city_key in cities:  # Use converted key with underscores
                city_info = cities[city_key]
                return {
                    'county': county_key.replace('_county', '').replace('_', ' ').title(),
                    'state_codes': city_info.get('state_codes', []),
                    'city_codes': city_info.get('city_codes', {}),
                    'building_codes': city_info.get('building_codes', {}),
                    'other_codes': city_info.get('other_codes', {}),
                    'found': True
                }
    
    # If city not found in any county, still return state codes (they apply statewide)
    return {
        'state_codes': ['AB_68', 'AB_881', 'SB_13']  # California state codes apply to all cities
    }

# Add ADU code information to the county_dfs
print("Adding ADU code information...")

# Create a single column with JSON format
county_dfs['adu_codes'] = ''

# Process each row to find ADU codes
for idx, row in county_dfs.iterrows():
    city_name = row['city']
    if pd.notna(city_name) and city_name.strip():
        adu_info = get_adu_codes_by_city(city_name, adu_ca_laws)
        # Store as JSON string
        county_dfs.at[idx, 'adu_codes'] = json.dumps(adu_info)
    else:
        # Even for empty city names, provide state codes
        county_dfs.at[idx, 'adu_codes'] = json.dumps({'state_codes': ['AB_68', 'AB_881', 'SB_13']})

# Display summary
print(f"\nADU Code Analysis Summary:")
print(f"Total properties: {len(county_dfs)}")

# Count properties with city-specific codes vs only state codes
city_specific_count = 0
state_only_count = 0

for idx, row in county_dfs.iterrows():
    adu_data = json.loads(row['adu_codes'])
    if 'city_codes' in adu_data or 'county' in adu_data:
        city_specific_count += 1
    else:
        state_only_count += 1

print(f"Properties with city-specific codes: {city_specific_count}")
print(f"Properties with state codes only: {state_only_count}")

# Show sample of the data


Adding ADU code information...

ADU Code Analysis Summary:
Total properties: 2563
Properties with city-specific codes: 1940
Properties with state codes only: 623

Sample properties with ADU codes:

City: San Jose
ADU Codes: {
  "county": "Santa Clara",
  "state_codes": [
    "AB_68",
    "AB_881",
    "AB_1033"
  ],
  "city_codes": {
    "Title_20": "Zoning Ordinance",
    "Section_20.30": "ADU Provisions",
    "PP23-007": "2024 Update"
  },
  "building_codes": {
    "SJMC_Title_24": "Building Code"
  },
  "other_codes": {
    "condominium_conversion": "AB-1033 implemented",
    "pre_approved_plans": "Available"
  },
  "found": true
}

City: San Jose
ADU Codes: {
  "county": "Santa Clara",
  "state_codes": [
    "AB_68",
    "AB_881",
    "AB_1033"
  ],
  "city_codes": {
    "Title_20": "Zoning Ordinance",
    "Section_20.30": "ADU Provisions",
    "PP23-007": "2024 Update"
  },
  "building_codes": {
    "SJMC_Title_24": "Building Code"
  },
  "other_codes": {
    "condominium_conversi

In [45]:
county_dfs

,address,city,zip,price,price_per_sqft,sqft,lot_sqft,beds,baths,description,...,url,lot_sqft_ratio,existing_adu,adu_county,adu_state_codes,adu_city_codes,adu_building_codes,adu_other_codes,adu_found,adu_codes
0,5850 Chesbro Ave,San Jose,95123,1499900,844.063028,1777.0,6000.0,3,3.0,"Welcome to 5850 Chesbro Ave, a beautifully remodeled home in the heart of Blossom Valley. This open-concept residence blends modern design with comfortable living, featuring a bright and spacious ...",...,https://www.realtor.com/realestateandhomes-detail/5850-Chesbro-Ave_San-Jose_CA_95123_M18665-35552,3.376477,0,Unknown,,,,,False,"{""county"": ""Santa Clara"", ""state_codes"": [""AB_68"", ""AB_881"", ""AB_1033""], ""city_codes"": {""Title_20"": ""Zoning Ordinance"", ""Section_20.30"": ""ADU Provisions"", ""PP23-007"": ""2024 Update""}, ""building_cod..."
1,1641 Rue Avati,San Jose,95131,1349888,1120.239004,1205.0,6247.0,3,2.0,"Welcome to 1641 Rue Avati, San Jose, CA 95131. This beautifully upgraded North San Jose home offers the perfect blend of modern comfort and unbeatable convenience. Featuring 3 bedrooms, 2 bathroom...",...,https://www.realtor.com/realestateandhomes-detail/1641-Rue-Avati_San-Jose_CA_95131_M14901-29936,5.184232,0,Unknown,,,,,False,"{""county"": ""Santa Clara"", ""state_codes"": [""AB_68"", ""AB_881"", ""AB_1033""], ""city_codes"": {""Title_20"": ""Zoning Ordinance"", ""Section_20.30"": ""ADU Provisions"", ""PP23-007"": ""2024 Update""}, ""building_cod..."
2,5991 Sorrel Ave,San Jose,95123,1600000,819.672131,1952.0,6120.0,5,2.0,"Welcome to 5991 Sorrel Avenue, a rare single-story, five-bedroom residence offering refined comfort and modern efficiency. With dual living spaces, tasteful updates, a beautifully landscaped backy...",...,https://www.realtor.com/realestateandhomes-detail/5991-Sorrel-Ave_San-Jose_CA_95123_M21370-78071,3.135246,0,Unknown,,,,,False,"{""county"": ""Santa Clara"", ""state_codes"": [""AB_68"", ""AB_881"", ""AB_1033""], ""city_codes"": {""Title_20"": ""Zoning Ordinance"", ""Section_20.30"": ""ADU Provisions"", ""PP23-007"": ""2024 Update""}, ""building_cod..."
3,266 Vista Ave,San Jose,95127,900000,813.008130,1107.0,7000.0,2,1.0,"PLEASE DO NOT DISTURB OCCUPANTS. Charming 2-bedroom home on a spacious 7, 000 sq. ft. lot in San Joses East Foothills. Prime location near Alum Rock and McKee with development potential, can possi...",...,https://www.realtor.com/realestateandhomes-detail/266-Vista-Ave_San-Jose_CA_95127_M23327-64684,6.323397,0,Unknown,,,,,False,"{""county"": ""Santa Clara"", ""state_codes"": [""AB_68"", ""AB_881"", ""AB_1033""], ""city_codes"": {""Title_20"": ""Zoning Ordinance"", ""Section_20.30"": ""ADU Provisions"", ""PP23-007"": ""2024 Update""}, ""building_cod..."
4,2936 Stutz Way,San Jose,95148,1650000,1057.692308,1560.0,6000.0,4,2.0,"Welcome to 2936 Stutz Way, a beautifully updated home in a great Evergreen neighborhood. This spacious floor plan offers four bedrooms, separate living and family rooms, a formal dining area, and ...",...,https://www.realtor.com/realestateandhomes-detail/2936-Stutz-Way_San-Jose_CA_95148_M29536-92966,3.846154,0,Unknown,,,,,False,"{""county"": ""Santa Clara"", ""state_codes"": [""AB_68"", ""AB_881"", ""AB_1033""], ""city_codes"": {""Title_20"": ""Zoning Ordinance"", ""Section_20.30"": ""ADU Provisions"", ""PP23-007"": ""2024 Update""}, ""building_cod..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2558,525 Hacienda Dr,Monrovia,91016,1125000,1057.330827,1064.0,7410.0,2,1.0,"Welcome to this charming 2 bedroom, 1 bathroom home nestled near the scenic Monrovia foothills. With 1, 064 square feet of comfortable living space, this residence offers a perfect blend of charac...",...,https://www.realtor.com/realestateandhomes-detail/525-Hacienda-Dr_Monrovia_CA_91016_M23894-90357,6.964286,0,Los Angeles,"AB_68, AB_881","Section_17.44.005: ADU and JADU Standards, 2025_Update: February 2025 revision",MMC_Title_15: Buildings,two_track_system: Bu